#### Mixing in Relationships

_Relationships_ created by `relationship()` are provided with _declarative mixin classes_ __exclusively__ using the `declared_attr` approach, __eliminating any ambiguity__ which could arise when _copying a relationship_ and its possibly _column-bound contents_. Below is an example which __combines a foreign key column and a relationship__ so that _two classes_ `Foo` and `Bar` can both be __configured to reference a common target class via many-to-one__.

In [1]:
from sqlalchemy import Column, Integer, ForeignKey
from sqlalchemy.orm import declared_attr, declarative_base, declarative_mixin, relationship

In [2]:
Base = declarative_base()

In [3]:
@declarative_mixin
class RefTargetMixin:
    @declared_attr
    def target_id(cls):
        return Column("target_id", ForeignKey("target.id"))
    
    @declared_attr
    def target(cls):
        return relationship("Target")

In [4]:
class Foo(RefTargetMixin, Base):
    __tablename__ = "foo"
    
    id = Column(Integer, primary_key=True)

In [5]:
class Bar(RefTargetMixin, Base):
    __tablename__ = "bar"
    
    id = Column(Integer, primary_key=True)

In [6]:
class Target(Base):
    __tablename__ = "target"
    
    id = Column(Integer, primary_key=True)

##### Using Advanced Relationship Arguments (e.g. primaryjoin, etc.)

`relationship()` definitions which require *explicit primaryjoin, order_by etc. expressions* should in _all but the most simplistic cases_ use __late bound forms__ for these arguments, meaning, using either the __string form or a function/lambda__. The reason for this is that the related `Column` objects which are to be __configured using `@declared_attr`__ are _not available to another_ `@declared_attr` attribute; while the _methods will work and return_ __new `Column` objects__, those are __not__ the `Column` objects that _Declarative_ will be using as it _calls the methods on its own_, thus using __different__ `Column` objects.

The _canonical_ example is the `primaryjoin` condition that __depends upon another mixed-in column__.

In [7]:
@declarative_mixin
class WrongParamRefTargetMixin:
    @declared_attr
    def target_id(cls):
        return Column("target_id", ForeignKey("target.id"))
    
    @declared_attr
    def target(cls):
        return relationship(Target, primaryjoin=Target.id == cls.target_id) # this is *incorrect*

_Mapping_ a class using the above _mixin_, we __will get an error__ like:

```
sqlalchemy.exc.InvalidRequestError: this ForeignKey's parent column is not
yet associated with a Table.
```

This is because the *target_id* `Column` we've called upon in our `target()` method is __not the same `Column`__ that _declarative_ is __actually going to map to our table__.

The condition above is _resolved_ using a `lambda`:

In [8]:
@declarative_mixin
class ParamRefTargetMixin:
    @declared_attr
    def target_id(cls):
        return Column("target_id", ForeignKey("target.id"))
    
    @declared_attr
    def target(cls):
        return relationship(Target, primaryjoin=lambda: Target.id == cls.target_id)

or alternatively, the `string form` (which _ultimately generates_ a `lambda`):

In [9]:
@declarative_mixin
class AlternativeParamRefTargetMixin:
    @declared_attr
    def target_id(cls):
        return Column("target_id", ForeignKey("target.id"))
    
    @declared_attr
    def target(cls):
        return relationship(Target, primaryjoin=f"Target.id == {cls.__name__}.target_id")